In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from Dataset.Economy_Dataset import EconomyDataset
from Dataset.Transaction_Dataset import TransactionDataset

from Model.LSTM import LSTM

In [2]:
input_size = 5
hidden_size = 256
output_size = 1

lr = 1e-3
num_epochs = 100

In [3]:
economy_df = pd.read_excel('../데이터/Economy/economy_all.xlsx')
economy_df = economy_df['국고채금리'].values
transaction_df = pd.read_csv('../데이터/Transaction/transaction_final.csv', index_col=0)

In [4]:
economy_train_size = int(len(economy_df)*0.6)
economy_val_size = int(len(economy_df)*0.3)
trainsaction_train_size = int(len(transaction_df)*0.6)
trainsaction_val_size = int(len(transaction_df)*0.3)

economy_train_dataset = EconomyDataset(economy_df[:economy_train_size])
economy_train_loader = DataLoader(economy_train_dataset, batch_size=2)
economy_val_dataset = EconomyDataset(economy_df[economy_train_size:economy_train_size+economy_val_size])
economy_val_loader = DataLoader(economy_val_dataset, batch_size=2)
economy_test_dataset = EconomyDataset(economy_df[economy_train_size+economy_val_size:])
economy_test_loader = DataLoader(economy_test_dataset, batch_size=2)

transaction_train_dataset = TransactionDataset(transaction_df[:trainsaction_train_size])
transaction_train_loader = DataLoader(transaction_train_dataset, batch_size=2)
transaction_val_dataset = TransactionDataset(transaction_df[trainsaction_train_size:trainsaction_train_size+trainsaction_val_size])
transaction_val_loader = DataLoader(transaction_val_dataset, batch_size=2)
transaction_test_dataset = TransactionDataset(transaction_df[trainsaction_train_size+trainsaction_val_size:])
transaction_test_loader = DataLoader(transaction_test_dataset, batch_size=2)

/Users/kimhakhyun/Desktop/Git/sci/SCI/코드/Dataset/Economy_Dataset.py:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  self.x = torch.FloatTensor(X)


In [5]:
model = LSTM(input_size, hidden_size, output_size)

In [6]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [7]:
# 경제 모델
best_val_loss = float('inf') 
for epoch in range(num_epochs + 1):
    model.train()
    for batch_idx, samples in enumerate(economy_train_loader):
        x_train, y_train = samples

        prediction = model(x_train)
        cost = criterion(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_idx, samples in enumerate(economy_val_loader):
            x_val, y_val = samples

            prediction = model(x_val)
            loss = criterion(prediction, y_val)
            val_loss += loss.item()

    val_loss /= len(economy_val_loader)
    print(f'Epoch {epoch}/{num_epochs}, Training Loss: {cost.item()}, Validation Loss: {val_loss}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '../코드/Checkpoint/best_economy_model.pth')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0/100, Training Loss: 0.584766149520874, Validation Loss: 1.7173492695604051
Epoch 1/100, Training Loss: 0.029341816902160645, Validation Loss: 0.4915290773829578
Epoch 2/100, Training Loss: 0.014836261048913002, Validation Loss: 0.673044593539089
Epoch 3/100, Training Loss: 0.011310496367514133, Validation Loss: 0.7112700086685696
Epoch 4/100, Training Loss: 0.00764611829072237, Validation Loss: 0.7607639563669052
Epoch 5/100, Training Loss: 0.005434147082269192, Validation Loss: 0.8015894385586891
Epoch 6/100, Training Loss: 0.0051312013529241085, Validation Loss: 0.8240756413766316
Epoch 7/100, Training Loss: 0.005548834800720215, Validation Loss: 0.8355885456715312
Epoch 8/100, Training Loss: 0.00619431771337986, Validation Loss: 0.8415670536591539
Epoch 9/100, Training Loss: 0.006885802838951349, Validation Loss: 0.844067028391042
Epoch 10/100, Training Loss: 0.0075322529301047325, Validation Loss: 0.8453299791019943
Epoch 11/100, Training Loss: 0.008100266568362713, Validat

In [8]:
# 부동산 모델
best_val_loss = float('inf') 
for epoch in range(num_epochs + 1):
    model.train()
    for batch_idx, samples in enumerate(transaction_train_loader):
        x_train, y_train = samples

        prediction = model(x_train)
        cost = criterion(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_idx, samples in enumerate(transaction_val_loader):
            x_val, y_val = samples

            prediction = model(x_val)
            loss = criterion(prediction, y_val)
            val_loss += loss.item()

    val_loss /= len(transaction_val_loader)
    print(f'Epoch {epoch}/{num_epochs}, Training Loss: {cost.item()}, Validation Loss: {val_loss}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '../코드/Checkpoint/best_transaction_model.pth')

KeyboardInterrupt: 